# Import Libraries

In [48]:
import pandas as pd 
import statistics
import matplotlib.pyplot as plt
import os 
import numpy as np
import gurobipy as gp
from gurobipy import GRB
m = gp.Model('dhl model')
import datetime

# Import Dataset

In [2]:
file_name = "odtu1data.xlsx"
os.chdir(r"C:\\Users\user\Desktop")
os.getcwd()

file = pd.ExcelFile(file_name)
data = file.parse(sheet_name="Collection Orders")

data.head()

,Order,W,L,H,Packing Type,Pallets Number,Gross Weight,Referance Number For Collection,Planned Date,Comment,Estimated Arrival date,Pallets Number.1,Weight,Departure Date,Plate Number
0,EDGBA/ Week1 /2019 / 4,120,100,60,Pallet,1.0,117.0,85288157,2019-01-03,NaN,2019-01-04,1,117,2019-01-04,34 BCP 800 / 34 ET 8505
1,E622D/ Week1 /2019 / 4,120,50,80,Pallet,1.0,342.0,TID 63195,2019-12-03,Not-Stackable,2019-01-07,1,471,2019-01-11,34 BAH 720 / 34 ZEJ 47
2,E622D/ Week1 /2019 / 4,80,50,60,Pallet,2.0,NaN,TID 63195,2019-12-03,Not-Stackable,2019-01-07,3,NaN,2019-01-11,34 BAH 720 / 34 ZEJ 47
3,R7GNA/ Week1 /2019 / 4,120,80,80,Pallet,2.0,790.0,ZF 2474789,2019-01-03,NaN,2019-01-07,2,798,2019-01-11,34 BAH 720 / 34 ZEJ 47
4,MWK9A/ Week1 /2019 / 4,60,80,45,Pallet,1.0,113.0,80040968,2019-01-03,Not-Stackable,2019-01-04,1,113,2019-01-04,34 BCP 800 / 34 ET 8505


# Data Exploration

In [3]:
type(data)

pandas.core.frame.DataFrame

In [9]:
data.shape

(2448, 16)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 15 columns):
Order                              2448 non-null object
W                                  2448 non-null int64
L                                  2448 non-null int64
H                                  2448 non-null int64
Packing Type                       2448 non-null object
Pallets Number                     2441 non-null float64
Gross Weight                       2448 non-null float64
Referance Number For Collection    1862 non-null object
Planned Date                       2448 non-null datetime64[ns]
Comment                            230 non-null object
Estimated Arrival date             2448 non-null datetime64[ns]
Pallets Number.1                   2448 non-null object
Weight                             1613 non-null object
Departure Date                     2448 non-null datetime64[ns]
Plate Number                       2448 non-null object
dtypes: datetime64[ns](3), floa

# Adjusting Empty Values

In [4]:
data['Gross Weight'].fillna(0, inplace=True)

# Volume Calculation

In [6]:
data['Volume']= data['W']*data['L']*data['H']*data['Pallets Number']/1000000
data.head()

,Order,W,L,H,Packing Type,Pallets Number,Gross Weight,Referance Number For Collection,Planned Date,Comment,Estimated Arrival date,Pallets Number.1,Weight,Departure Date,Plate Number,Volume
0,EDGBA/ Week1 /2019 / 4,120,100,60,Pallet,1.0,117.0,85288157,2019-01-03,NaN,2019-01-04,1,117,2019-01-04,34 BCP 800 / 34 ET 8505,0.720
1,E622D/ Week1 /2019 / 4,120,50,80,Pallet,1.0,342.0,TID 63195,2019-12-03,Not-Stackable,2019-01-07,1,471,2019-01-11,34 BAH 720 / 34 ZEJ 47,0.480
2,E622D/ Week1 /2019 / 4,80,50,60,Pallet,2.0,0.0,TID 63195,2019-12-03,Not-Stackable,2019-01-07,3,NaN,2019-01-11,34 BAH 720 / 34 ZEJ 47,0.480
3,R7GNA/ Week1 /2019 / 4,120,80,80,Pallet,2.0,790.0,ZF 2474789,2019-01-03,NaN,2019-01-07,2,798,2019-01-11,34 BAH 720 / 34 ZEJ 47,1.536
4,MWK9A/ Week1 /2019 / 4,60,80,45,Pallet,1.0,113.0,80040968,2019-01-03,Not-Stackable,2019-01-04,1,113,2019-01-04,34 BCP 800 / 34 ET 8505,0.216


In [8]:
data.columns.values

array(['Order', 'W', 'L', 'H', 'Packing Type', 'Pallets Number',
       'Gross Weight', 'Referance Number For Collection', 'Planned Date',
       'Comment', 'Estimated Arrival date', 'Pallets Number.1', 'Weight',
       'Departure Date', 'Plate Number', 'Volume'], dtype=object)

# Splitting Day, Week, Year Values

In [50]:
#data[['Supplier','Weeknum','Daynum']]
a = data['Order'].str.split('/',expand=True)
data['Supplier'] = a[0]
data['Weeknum'] = a[1]
data['Year'] = a[2]
data['Daynum'] = a[3]
b = data['Weeknum'].str.split('k',expand=True)
data['# of week'] = b[1]
data['# of week'] = pd.to_numeric(data['# of week'])
data['Daynum'] = pd.to_numeric(data['Daynum'])
data['Year'] = pd.to_numeric(data['Year'])
type(data['Year'][4])

numpy.int64

# Day of Year Calculation

In [56]:
data['Day of Year'] = (data['# of week'] -1)*7  +  data['Daynum']

In [60]:
data.head()

,Order,W,L,H,Packing Type,Pallets Number,Gross Weight,Referance Number For Collection,Planned Date,Comment,...,Weight,Departure Date,Plate Number,Volume,Supplier,Weeknum,Daynum,Year,# of week,Day of Year
0,EDGBA/ Week1 /2019 / 4,120,100,60,Pallet,1.0,117.0,85288157,2019-01-03,NaN,...,117,2019-01-04,34 BCP 800 / 34 ET 8505,0.720,EDGBA,Week1,4.0,2019,1.0,4.0
1,E622D/ Week1 /2019 / 4,120,50,80,Pallet,1.0,342.0,TID 63195,2019-12-03,Not-Stackable,...,471,2019-01-11,34 BAH 720 / 34 ZEJ 47,0.480,E622D,Week1,4.0,2019,1.0,4.0
2,E622D/ Week1 /2019 / 4,80,50,60,Pallet,2.0,0.0,TID 63195,2019-12-03,Not-Stackable,...,NaN,2019-01-11,34 BAH 720 / 34 ZEJ 47,0.480,E622D,Week1,4.0,2019,1.0,4.0
3,R7GNA/ Week1 /2019 / 4,120,80,80,Pallet,2.0,790.0,ZF 2474789,2019-01-03,NaN,...,798,2019-01-11,34 BAH 720 / 34 ZEJ 47,1.536,R7GNA,Week1,4.0,2019,1.0,4.0
4,MWK9A/ Week1 /2019 / 4,60,80,45,Pallet,1.0,113.0,80040968,2019-01-03,Not-Stackable,...,113,2019-01-04,34 BCP 800 / 34 ET 8505,0.216,MWK9A,Week1,4.0,2019,1.0,4.0


# Aggregating Volume, Weight & Displaying Order Date Based on Aggregate Values

In [69]:
tablo = data.groupby('Order')['Gross Weight','Volume'].sum()
type(tablo)

pandas.core.frame.DataFrame

In [63]:
hacim = data.groupby('Order')['Volume'].sum()

In [72]:
siparis_tarih = data.groupby(['Order'], sort=False)['Day of Year'].max()
siparis_tarih

Order
EDGBA/ Week1 /2019 / 4       4.0
E622D/ Week1 /2019 / 4       4.0
R7GNA/ Week1 /2019 / 4       4.0
MWK9A/ Week1 /2019 / 4       4.0
FZG8A/ Week1 /2019 / 4       4.0
                           ...  
GU40A/ Week51 /2019 / 2    352.0
EDGBA/ Week51 /2019 / 2    352.0
G944A/ Week51 /2019 / 2    352.0
DYRZA/ Week50 /2019 / 3    346.0
F470A/ Week51/2019 / 3     353.0
Name: Day of Year, Length: 1607, dtype: float64

In [75]:
tedarikci = data.groupby(['Order'], sort=False)['Supplier'].max()

In [76]:
pd.concat([tablo, siparis_tarih, tedarikci], axis=1)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Gross Weight,Volume,Day of Year,Supplier
A426R/ Week1 /2019 / 1,2016.0,3.60,1.0,A426R
A426R/ Week10 /2019 / 3,2688.0,4.80,66.0,A426R
A426R/ Week11 /2019 / 3,3360.0,6.00,73.0,A426R
A426R/ Week12 /2019 / 3,2688.0,4.80,80.0,A426R
A426R/ Week13 /2019 / 3,2688.0,4.80,87.0,A426R
...,...,...,...,...
U2W5B/ Week27 /2019 / 3,423.0,0.48,185.0,U2W5B
U2W5B/ Week30 /2019 / 2,1104.0,1.44,205.0,U2W5B
U2W5B/ Week37 /2019 / 5,1349.0,1.92,257.0,U2W5B
U2W5B/ Week6 /2019 / 1,423.0,0.48,36.0,U2W5B


In [77]:
#datetime.datetime(2019, 1, 1) + datetime.timedelta(data['Day of Year'])

In [ ]:
#(np.asarray(data['Year'], dtype='datetime64[Y]')-1970)+(np.asarray(df['doy'], dtype='timedelta64[D]')-1)

In [55]:
pd.to_datetime(data['Year'] * 1000 + data['Daynum'], format='%Y%j')

0      2019-01-04
1      2019-01-04
2      2019-01-04
3      2019-01-04
4      2019-01-04
          ...    
2443   2019-01-02
2444   2019-01-02
2445   2019-01-02
2446   2019-01-03
2447   2019-01-03
Length: 2448, dtype: datetime64[ns]